In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import post_patch as pp #post patch script
#post_patch_column = 'postPatch_label'
#post_patch_pipette_column = 'endPipetteR_label'

sns.set(style = "whitegrid")
sns.set_context('notebook')

### Creating Dataframes: json_df, morph_df, merged_df

In [2]:
def get_json_df():
    json_df = pd.read_csv('C:/users/kumar/documents/github/personal-projects/csv/json_metadata.csv')
    json_df = json_df[json_df['status'] == 'SUCCESS']

    #Creates patch_duration column and outputs in minutes
    json_df['patch_duration'] = pd.to_datetime(json_df['extraction.timeRetractionEnd']) - pd.to_datetime(json_df['recording.timeWholeCellStart'])
    json_df['patch_duration'] = (json_df['patch_duration'].astype('timedelta64[s]'))/60

    json_df['date'] = json_df['date'].str[:10] #Strip away the time
    json_df['date'] = pd.to_datetime(json_df['date']) #Converting to YYYY-MM-DD

    json_df = pp.postpatch_reclass(json_df) #Makes post patch column with the script

    json_df.dropna(subset = ['patch_container', 'cell_depth'], inplace = True)
    json_df = json_df[json_df['patch_container'].str.match('PA')]

    json_df = json_df[['date', 'patch_container', 'patch_duration', 'post_patch']]
    json_df.sort_values(by = ['date'], inplace = True) #Sorts date column
    return json_df

In [3]:
def get_morph_df():
    morph_df = pd.read_csv("C:/users/kumar/documents/github/personal-projects/csv/63x_mouse.csv")
    morph_df = morph_df.loc[:,['patched_cell_container', 
                               'ims63x_go_no_go', 
                               'ims63x_imaging_started', 
                               'image_series_63x_qc',
                               'link_ims_20x',
                               'link_ims_63x']]
    return morph_df

In [4]:
def merge_dfs(df1, df2):
    merged_df = pd.merge(left = df1, 
                         right = df2, 
                         left_on = 'patch_container',
                         right_on = 'patched_cell_container',
                         how = 'inner')
    merged_df.dropna(subset=['ims63x_go_no_go'], inplace = True)
    merged_df.drop(columns=['patched_cell_container'], inplace=True)
    merged_df.sort_values(by = ['date'], inplace = True) #Sorts date column
    return merged_df

In [5]:
#Making global variables
json_df = get_json_df()
morph_df = get_morph_df()
merged_df = merge_dfs(json_df, morph_df)

C:\Users\kumar\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\kumar\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Dataframe Info

In [6]:
json_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 785 entries, 223 to 6463
Data columns (total 4 columns):
date               785 non-null datetime64[ns]
patch_container    785 non-null object
patch_duration     785 non-null float64
post_patch         785 non-null object
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 30.7+ KB


In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406 entries, 0 to 657
Data columns (total 9 columns):
date                      406 non-null datetime64[ns]
patch_container           406 non-null object
patch_duration            406 non-null float64
post_patch                406 non-null object
ims63x_go_no_go           406 non-null object
ims63x_imaging_started    207 non-null object
image_series_63x_qc       198 non-null object
link_ims_20x              406 non-null object
link_ims_63x              198 non-null object
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 31.7+ KB


In [8]:
merged_df['post_patch'].value_counts()

Nuc-high seal    344
Nuc-low seal      36
No-low seal       14
Entire cell        7
No-high seal       5
Name: post_patch, dtype: int64

In [9]:
merged_df['ims63x_go_no_go'].value_counts()

63x go       213
63x no go    193
Name: ims63x_go_no_go, dtype: int64

### Display df

In [10]:
json_df.head()

,date,patch_container,patch_duration,post_patch
223,2017-10-02,PAS4_171002_451_A01,11.866667,No-low seal
242,2017-10-03,PAS4_171003_451_A01,14.483333,Nuc-high seal
266,2017-10-03,PAS4_171003_452_A01,12.500000,Nuc-high seal
294,2017-10-04,PAS4_171004_452_A01,9.633333,Nuc-high seal
303,2017-10-04,PAS4_171004_453_A01,14.350000,Nuc-high seal


In [11]:
json_df.tail()

,date,patch_container,patch_duration,post_patch
6440,2019-01-11,PAS4_190111_457_A01,9.966667,Nuc-high seal
6475,2019-01-15,PAS4_190115_452_A01,7.650000,Nuc-high seal
6476,2019-01-15,PAS4_190115_454_A01,12.300000,Nuc-high seal
6477,2019-01-15,PAS4_190115_453_A01,12.383333,Nuc-high seal
6463,2019-01-15,PAS4_190115_451_A01,9.600000,Nuc-high seal


In [12]:
merged_df.head()

,date,patch_container,patch_duration,post_patch,ims63x_go_no_go,ims63x_imaging_started,image_series_63x_qc,link_ims_20x,link_ims_63x
0,2017-10-02,PAS4_171002_451_A01,11.866667,No-low seal,63x no go,NaN,NaN,http://lims2/focal_plane_image_series?id=64349...,NaN
1,2017-10-03,PAS4_171003_451_A01,14.483333,Nuc-high seal,63x no go,NaN,NaN,http://lims2/focal_plane_image_series?id=64349...,NaN
2,2017-10-03,PAS4_171003_452_A01,12.500000,Nuc-high seal,63x go,63x imaging started,image_qc_passed,http://lims2/focal_plane_image_series?id=64349...,http://lims2/focal_plane_image_series?id=64726...
3,2017-10-04,PAS4_171004_452_A01,9.633333,Nuc-high seal,63x go,63x imaging started,ready_for_dendrite_trace,http://lims2/focal_plane_image_series?id=64350...,http://lims2/focal_plane_image_series?id=65053...
4,2017-10-04,PAS4_171004_453_A01,14.350000,Nuc-high seal,63x go,63x imaging started,deferred,http://lims2/focal_plane_image_series?id=64350...,http://lims2/focal_plane_image_series?id=66583...


In [13]:
merged_df.tail()

,date,patch_container,patch_duration,post_patch,ims63x_go_no_go,ims63x_imaging_started,image_series_63x_qc,link_ims_20x,link_ims_63x
651,2018-11-15,PAS4_181115_453_A01,16.100000,Nuc-high seal,63x go,NaN,NaN,http://lims2/focal_plane_image_series?id=78999...,NaN
652,2018-11-15,PAS4_181115_451_A01,12.166667,Nuc-high seal,63x go,NaN,NaN,http://lims2/focal_plane_image_series?id=78999...,NaN
654,2018-11-15,PAS4_181115_455_A01,12.533333,Nuc-high seal,63x no go,NaN,NaN,http://lims2/focal_plane_image_series?id=78999...,NaN
656,2018-11-16,PAS4_181116_452_A01,7.000000,Nuc-high seal,63x no go,NaN,NaN,http://lims2/focal_plane_image_series?id=78999...,NaN
657,2018-11-16,PAS4_181116_451_A01,13.566667,Nuc-high seal,63x no go,NaN,NaN,http://lims2/focal_plane_image_series?id=78999...,NaN


In [14]:
#multi_df = merged_df.set_index(['ims63x_go_no_go', 'post_patch']) #Multi level index

In [15]:
#Outputing dataframe as excel sheet
#writer = pd.ExcelWriter('TITLE.xlsx')
#dataframe.to_excel(writer, 'Sheet#)

writer = pd.ExcelWriter('morph_df.xlsx')
merged_df.to_excel(writer, 'Sheet1')
writer.save()